# REDDY

Load modules and initialize objects

In [1]:
from datetime import datetime, timedelta
from progressbar import ProgressBar, SimpleProgress, ETA
from modules.reddit_api import RedditAPI
from modules.cache_handler import Cache
from credentials import client_id, secret

api   = RedditAPI()
cache = Cache(verbose=False)

api.authenticate(client_id, secret)

# name of the subreddit to analyse
subreddit_name = "datascience"

Download the posts and index them

In [2]:
after = None
count = 0
pbar = ProgressBar(widgets=[SimpleProgress(), ', ', ETA(), "\n"])
iterations = pbar(range(10))

for i in iterations:
    posts, before, after = api.new_posts(subreddit_name, limit=100, after=after, count=count)
    count += len(posts)
    cache.add(posts, overwrite=False)

cache.update_all_indices()

0 of 10, ETA:  --:--:--
1 of 10, ETA:  0:00:09
2 of 10, ETA:  0:00:08
3 of 10, ETA:  0:00:07
4 of 10, ETA:  0:00:07
5 of 10, ETA:  0:00:05
6 of 10, ETA:  0:00:04
7 of 10, ETA:  0:00:03
8 of 10, ETA:  0:00:02
9 of 10, ETA:  0:00:01
10 of 10, Time: 0:00:12



In [4]:
lastposts = cache.select(subreddit_name, t=(datetime.utcnow()-timedelta(days=1), datetime.utcnow()))

print(lastposts)

post_id       created   created_utc   ups  downs view_count
26   gjio4i  1.589476e+09  1.589447e+09   1.0    0.0       None
45   gjd820  1.589451e+09  1.589422e+09  75.0    0.0       None
71   gjk954  1.589483e+09  1.589454e+09   1.0    0.0       None
517  gj3i6j  1.589420e+09  1.589391e+09   2.0    0.0       None
639  gjj7rx  1.589478e+09  1.589449e+09   3.0    0.0       None
641  gjkajt  1.589483e+09  1.589455e+09   1.0    0.0       None
694  gjkyy3  1.589486e+09  1.589458e+09   1.0    0.0       None
718  giz5ve  1.589406e+09  1.589377e+09   4.0    0.0       None
789  gjhhe2  1.589470e+09  1.589441e+09   1.0    0.0       None
850  gjl45g  1.589487e+09  1.589458e+09   1.0    0.0       None
918  gj4687  1.589422e+09  1.589393e+09   3.0    0.0       None
963  gjl0sf  1.589487e+09  1.589458e+09   1.0    0.0       None
